This file can be submitted to Kaggle and it will successfully submit an attempt

In [ ]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
import datetime # ...

In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 50
NCOLS = 15
# Main data location path...
BASE_PATH = '...'

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.5f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

In [ ]:
import gc
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OneHotEncoder, OrdinalEncoder
def get_data(fill_values  = [3, 2, 0, 3, 0], get_test=False):
    # Load the CSV information into a Pandas DataFrame...
    trn_data = pd.read_feather('../input/parquet-files-amexdefault-prediction/train_data.ftr')
    trn_lbls = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv').set_index('customer_ID')
    if(get_test):
        tst_data = pd.read_feather('../input/parquet-files-amexdefault-prediction/test_data.ftr')

    #%%time
    if(get_test):
        sub = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
    else:
        sub = "Get test off"
    ## 6.1 Training Dataset...
    print(trn_data[:5])
    # We have 458913 customers. and we have 458913 train labels...

    # Calculates the amount of information by costumer or records available...
    trn_num_statements = trn_data.groupby('customer_ID').size().sort_index()
    print(trn_data)
    # Create a new dataset based on aggregated information
    trn_agg_data = (trn_data
                    .groupby('customer_ID')
                    .tail(1)
                    .set_index('customer_ID', drop=True)
                    .sort_index()
                    .drop(['S_2'], axis='columns'))
    del trn_data
    # Merge the labels from the labels dataframe
    trn_agg_data['target'] = trn_lbls.target
    del trn_lbls
    trn_agg_data['num_statements'] = trn_num_statements
    del trn_num_statements
    
    trn_agg_data.reset_index(inplace = True, drop = True) # forget the customer_IDs

    ## 6.2 Test Dataset...

    # Calculates the amount of information by costumer or records available...
    if(get_test):
        tst_num_statements = tst_data.groupby('customer_ID').size().sort_index()

        # Create a new dataset based on aggregated information
        tst_agg_data = (tst_data
                        .groupby('customer_ID')
                        .tail(1)
                        .set_index('customer_ID', drop=True)
                        .sort_index()
                        .drop(['S_2'], axis='columns'))
        del tst_data
        # Merge the labels from the labels dataframe
        tst_agg_data['num_statements'] = tst_num_statements
        del tst_num_statements
        tst_agg_data.reset_index(inplace = True, drop = True) # forget the customer_IDs

    # 7.0 Label / One-Hot Encoding the Categorical Variables...

    ## 7.1 One Hot Encoding Configuration...

    # One-hot Encoding Configuration
    cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

    #trn_agg_data[cat_features] = trn_agg_data[cat_features].astype(object)
    trn_not_cat_features = [f for f in trn_agg_data.columns if f not in cat_features]
    if(get_test):
        tst_not_cat_features = [f for f in tst_agg_data.columns if f not in cat_features]

    #encoder = OneHotEncoder(drop = 'first', sparse = False, dtype = np.float32, handle_unknown = 'ignore')
    encoder = OrdinalEncoder()
    trn_encoded_features = encoder.fit_transform(trn_agg_data[cat_features])
    #feat_names = list(encoder.get_feature_names())

    ## 7.2 Train Dataset One Hot Encoding...

    # One-hot Encoding
    trn_encoded_features = pd.DataFrame(trn_encoded_features)
    #trn_encoded_features.columns = feat_names

    trn_agg_data = pd.concat([trn_agg_data[trn_not_cat_features], trn_encoded_features], axis = 1)

    ## 7.3 Test Dataset One-Hot Encoding...
    if(get_test):
        # One-hot Encoding
        tst_encoded_features = encoder.transform(tst_agg_data[cat_features])
        tst_encoded_features = pd.DataFrame(tst_encoded_features)
        #tst_encoded_features.columns = feat_names

        tst_agg_data = pd.concat([tst_agg_data[tst_not_cat_features], tst_encoded_features], axis = 1)
        tst_agg_data.head()

    features = [f for f in trn_agg_data.columns if f != 'target' and f != 'customer_ID']
    
    c = trn_agg_data[features].columns.str
    cs = [c.startswith('S_', False), c.startswith('P_', False), c.startswith('B_', False), c.startswith('R_', False), c.startswith('D_', False)]
    cs = [trn_agg_data[features].columns[c_i] for c_i in cs]
    print(cs)
    #
    # Impute missing values
    # Old fill type values
    for i_fill in range(len(fill_values)):
        if(fill_values[i_fill]==0):
            trn_agg_data[cs[i_fill]].fillna(value = 0, inplace = True)
            if(get_test):
                tst_agg_data[cs[i_fill]].fillna(value = 0, inplace = True)
        elif(fill_values[i_fill]==1):
            trn_agg_data[cs[i_fill]].fillna(value = np.nanmean(trn_agg_data[cs[i_fill]]), inplace = True)
            if(get_test):
                tst_agg_data[cs[i_fill]].fillna(value = np.nanmean(trn_agg_data[cs[i_fill]]), inplace = True)         
        elif(fill_values[i_fill]==2):
            trn_agg_data[cs[i_fill]].fillna(value = np.nanquantile(trn_agg_data[cs[i_fill]], .25), inplace = True)
            if(get_test):
                tst_agg_data[cs[i_fill]].fillna(value = np.nanquantile(trn_agg_data[cs[i_fill]], .25), inplace = True)       
        elif(fill_values[i_fill]==3):
            trn_agg_data[cs[i_fill]].fillna(value = np.nanquantile(trn_agg_data[cs[i_fill]], .75), inplace = True)
            if(get_test):
                tst_agg_data[cs[i_fill]].fillna(value = np.nanquantile(trn_agg_data[cs[i_fill]], .75), inplace = True)            
    #Fill all others
    trn_agg_data.fillna(value = 0, inplace = True)
    if(get_test):
        tst_agg_data.fillna(value = 0, inplace = True)

    # 10.0 NN Development

    # Release some memory by deleting the original DataFrames...
    gc.collect()
    if(get_test==False):
        tst_agg_data = 0
    return trn_agg_data, tst_agg_data, features, sub

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout, Concatenate, Reshape, Conv1D, Flatten
from tensorflow.keras.utils import plot_model
from sklearn.metrics import log_loss

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

In [ ]:
%%time
# From https://www.kaggle.com/code/inversion/amex-competition-metric-python
import torch

def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    
    
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    
    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    
    df = pd.DataFrame({'target': tf.experimental.numpy.ravel(y_true), 'prediction': tf.experimental.numpy.ravel(y_pred)})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [ ]:
%%time
def nn_model():
    '''
    '''
    regularization = 4e-4
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(regularization), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    
    x = Dense(64, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(regularization), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    
    x = Dense(64, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(regularization), 
          activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(32, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(regularization), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(1, 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(regularization),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

In [ ]:
%%time
# Defining model parameters...
BATCH_SIZE         = 256
EPOCHS             = 170 
EPOCHS_COSINEDECAY = 170
DIAGRAMS           = True
USE_PLATEAU        = False
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'target'

In [ ]:
%%time
def nn_model(features, regularization = 4e-4):
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))

    x0 = Dense(256,
               kernel_regularizer = tf.keras.regularizers.l2(regularization), 
               activation = activation_func)(inputs)
    x1 = Dense(128,
               kernel_regularizer = tf.keras.regularizers.l2(regularization),
               activation = activation_func)(x0)
    x1 = Dense(64,
               kernel_regularizer = tf.keras.regularizers.l2(regularization),
               activation = activation_func)(x1)
    x1 = Dense(32,
           kernel_regularizer = tf.keras.regularizers.l2(regularization),
           activation = activation_func)(x1)
    
    x1 = Concatenate()([x1, x0])
    x1 = Dropout(0.1)(x1)
    
    x1 = Dense(16, kernel_regularizer=tf.keras.regularizers.l2(regularization),activation=activation_func,)(x1)
    
    x1 = Dense(1, 
              #kernel_regularizer=tf.keras.regularizers.l2(regularization),
              activation='sigmoid')(x1)
    
    model = Model(inputs, x1)
    
    return model
    

In [ ]:
 %%time
# Defining model training function...
history_list = []
def fit_model(X_train, y_train, X_val, y_val, model, run = 0):
    '''
    '''
    lr_start = 0.01
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 4, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 12, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
    
    # Model Initialization...
    #model = nn_model(features)
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func, metrics = [amex_metric])
    
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    print("Model fitted")
    history_list.append(history.history)
        
    print(f'Training Loss: {history_list[-1]["loss"][-1]:.5f}, Validation Loss: {history_list[-1]["val_loss"][-1]:.5f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE).ravel()
    amex_score = amex_metric(y_val.values, y_val_pred, return_components = False)
    
    print(f'Fold {run} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| Amex Score: {amex_score:.5f}')
    
    print('')
    
    #score_list.append(amex_score)
    
    tst_data_scaled = scaler.transform(tst_agg_data[features])
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    print(amex_score)
    print(history)
    return amex_score, history.history

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import math
def train(trn_agg_data, features, model):
    score_list = []
    kf = KFold(n_splits = 5)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_agg_data)):
        X_train, X_val = trn_agg_data.iloc[trn_idx][features], trn_agg_data.iloc[val_idx][features]
        y_train, y_val = trn_agg_data.iloc[trn_idx][TARGET], trn_agg_data.iloc[val_idx][TARGET]
        print("Fold",fold)
        score, history = fit_model(X_train, y_train, X_val, y_val, model)
        print("In loop (train):", history)
        score_list.append(score)
    current_score = np.mean(score_list)
    print(f'OOF AUC: {current_score:.5f}')
    print(history)
    return history

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
%%time
gc.collect()
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import math

# Create empty lists to store NN information...
trn_agg_data, tst_agg_data, features, sub = get_data(get_test=True)

In [ ]:
score_list = []
predictions = []
history = train(trn_agg_data, features, nn_model(features))

In [ ]:
history_list

In [ ]:
%%time
sub.head()

In [ ]:
%%time
sub['prediction'] = np.array(predictions).mean(axis = 0)

In [ ]:
%%time
sub.to_csv('my_submission.csv', index = False)

In [ ]:
%%time
sub.head()